In [2]:
# All'inizio del notebook
import sys
import os

# Aggiungi il root del progetto al Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from wavelet_lib.datasets import TileDataset
from wavelet_lib.models import TileWaveletClassifier
from torch.utils.data import Subset
from wavelet_lib.processors import DroneImageWaveletProcessor
from wavelet_lib.base import BaseWaveletDataset, WaveletProcessor
import pickle
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import os



In [3]:
# Configurazione
SCATTERING_PARAMS = {
    "J": 2,
    "shape": (32, 32),
    "max_order": 2
}

In [4]:
# Trasformazioni
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
processor = DroneImageWaveletProcessor(J=2, shape=(32, 32))
print(f"Processor initialized with shape {processor.shape}")

print("\nCreating dataset...")

# Dataset e DataLoader
dataset = TileDataset(
    dataset_root="/home/brus/Projects/wavelet/datasets/HPL_images/custom_dataset",
    processor=processor,
    transform=transform,
    balance=True,
    allowed_extensions={'.jpg'}
)
first_data, _ = dataset[0]
print(f"First item shape: {first_data.shape}")

print(f"\nDataset creation completed:")
print(f"Total samples: {len(dataset)}")
print(f"Number of classes: {len(dataset.classes)}")
print(f"Classes: {dataset.classes}")

# Salvataggio del dataset elaborato
import pickle
import os

# Crea la directory per i dataset elaborati se non esiste
save_dir = "/home/brus/Projects/wavelet/datasets/wavelet_datasets"
os.makedirs(save_dir, exist_ok=True)

# Nome del file di salvataggio
save_path = os.path.join(save_dir, "processed_tile_dataset.pkl")

# Salva il dataset
with open(save_path, 'wb') as f:
    pickle.dump({
        'data': dataset.data,
        'wavelet_representations': dataset.wavelet_representations,
        'samples': dataset.samples,
        'classes': dataset.classes,
        'class_to_idx': dataset.class_to_idx
    }, f)

print(f"\nDataset salvato in: {save_path}")

Processor initialized with shape (32, 32)

Creating dataset...

Loading raw data...
Raw data loading completed!

Preparing wavelet representations...
Wavelet representations completed!

Loading raw data...
Loading file 100/39690 (0.3%)
Loading file 200/39690 (0.5%)
Loading file 300/39690 (0.8%)
Loading file 400/39690 (1.0%)
Loading file 500/39690 (1.3%)
Loading file 600/39690 (1.5%)
Loading file 700/39690 (1.8%)
Loading file 800/39690 (2.0%)
Loading file 900/39690 (2.3%)
Loading file 1000/39690 (2.5%)
Loading file 1100/39690 (2.8%)
Loading file 1200/39690 (3.0%)
Loading file 1300/39690 (3.3%)
Loading file 1400/39690 (3.5%)
Loading file 1500/39690 (3.8%)
Loading file 1600/39690 (4.0%)
Loading file 1700/39690 (4.3%)
Loading file 1800/39690 (4.5%)
Loading file 1900/39690 (4.8%)
Loading file 2000/39690 (5.0%)
Loading file 2100/39690 (5.3%)
Loading file 2200/39690 (5.5%)
Loading file 2300/39690 (5.8%)
Loading file 2400/39690 (6.0%)
Loading file 2500/39690 (6.3%)
Loading file 2600/39690 (6.6

KeyboardInterrupt: 

In [5]:
# carica dataset da pickle

class LoadedTileDataset(Dataset):
    def __init__(self, pickle_path):
        # Carica il dataset salvato
        print(f"Caricamento dataset da {pickle_path}")
        with open(pickle_path, 'rb') as f:
            saved_data = pickle.load(f)
            
        # Estrai i componenti
        self.data = saved_data['data']
        self.wavelet_representations = saved_data['wavelet_representations']
        self.samples = saved_data['samples']
        self.classes = saved_data['classes']
        self.class_to_idx = saved_data['class_to_idx']
        
        print("\nInformazioni Dataset:")
        print(f"Numero totale campioni: {len(self.samples)}")
        print(f"Numero classi: {len(self.classes)}")
        print(f"Classi disponibili: {self.classes}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        filepath, label = self.samples[idx]
        # Usa la rappresentazione wavelet pre-calcolata
        image = self.wavelet_representations[filepath]
        return image, label

# Carica il dataset
pickle_path = "/home/brus/Projects/wavelet/datasets/processed_datasets/processed_tile_dataset.pkl"
dataset = LoadedTileDataset(pickle_path)


Caricamento dataset da /home/brus/Projects/wavelet/datasets/processed_datasets/processed_tile_dataset.pkl

Informazioni Dataset:
Numero totale campioni: 39690
Numero classi: 7
Classi disponibili: ['human structure', 'mudflat', 'street', 'vegetation 1', 'vegetation 2', 'vegetation 3', 'water']


In [6]:
train_indices, test_indices = train_test_split(
    range(len(dataset)), 
    test_size=0.2, 
    random_state=42, 
    stratify=[label for _, label in dataset.samples]
)

In [ ]:
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TileWaveletClassifier(
    scattering_params=SCATTERING_PARAMS
).to(device)

In [9]:
def print_configuration_summary(dataset, train_loader, test_loader, model):
    """
    Stampa un riepilogo dettagliato della configurazione del sistema.
    """    
    print("\n" + "="*80)
    print(" "*30 + "RIEPILOGO CONFIGURAZIONE" + " "*30)
    print("="*80)

    # Informazioni sul dataset
    print("\n[1] DATASET:")
    print(f"  • Numero di classi: {len(dataset.classes)}")
    print(f"  • Classi: {dataset.classes}")
    print(f"  • Campioni totali: {len(dataset)}")
    print(f"  • Split: {len(train_loader.dataset)} training / {len(test_loader.dataset)} test")
    
    # Informazioni sulle rappresentazioni wavelet
    print("\n[2] RAPPRESENTAZIONI WAVELET:")
    sample_key = list(dataset.wavelet_representations.keys())[0]
    sample_repr = dataset.wavelet_representations[sample_key]
    print(f"  • Shape rappresentazione: {sample_repr.shape}")
    print(f"  • Tipo dati: {sample_repr.dtype}")
    print(f"  • Numero totale rappresentazioni: {len(dataset.wavelet_representations)}")
    
    # Informazioni sul modello
    print("\n[3] MODELLO:")
    print(f"  • Architettura: {model.__class__.__name__}")
    print(f"  • Parametri totali: {sum(p.numel() for p in model.parameters()):,}")
    print(f"  • Parametri allenabili: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    print(f"  • Device: {next(model.parameters()).device}")

    # Informazioni sui dataloader
    print("\n[4] DATALOADER:")
    print(f"  • Batch size training: {train_loader.batch_size}")
    print(f"  • Batch size test: {test_loader.batch_size}")
    print(f"  • Training batches: {len(train_loader)}")
    print(f"  • Test batches: {len(test_loader)}")
    print(f"  • Num workers: {train_loader.num_workers}")

    # Informazioni sul sistema
    print("\n[5] SISTEMA:")
    print(f"  • PyTorch version: {torch.__version__}")
    print(f"  • CUDA disponibile: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  • Dispositivo CUDA: {torch.cuda.get_device_name(0)}")
        print(f"  • Memoria totale GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print(f"  • Memoria allocata GPU: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"  • CPU count: {os.cpu_count()}")

    print("\n" + "="*80)
    print(" "*25 + "CONFIGURAZIONE PRONTA PER L'ADDESTRAMENTO" + " "*25)
    print("="*80 + "\n")

# Chiamata alla funzione
print_configuration_summary(dataset, train_loader, test_loader, model)


                              RIEPILOGO CONFIGURAZIONE                              

[1] DATASET:
  • Numero di classi: 7
  • Classi: ['human structure', 'mudflat', 'street', 'vegetation 1', 'vegetation 2', 'vegetation 3', 'water']
  • Campioni totali: 39690
  • Split: 31752 training / 7938 test

[2] RAPPRESENTAZIONI WAVELET:
  • Shape rappresentazione: torch.Size([3, 81, 8, 8])
  • Tipo dati: torch.float32
  • Numero totale rappresentazioni: 39690

[3] MODELLO:
  • Architettura: TileWaveletClassifier
  • Parametri totali: 21,282,317
  • Parametri allenabili: 21,282,317
  • Device: cuda:0

[4] DATALOADER:
  • Batch size training: 128
  • Batch size test: 128
  • Training batches: 249
  • Test batches: 63
  • Num workers: 4

[5] SISTEMA:
  • PyTorch version: 2.6.0+cu124
  • CUDA disponibile: True
  • Dispositivo CUDA: NVIDIA GeForce RTX 3050 Laptop GPU
  • Memoria totale GPU: 3.97 GB
  • Memoria allocata GPU: 0.09 GB
  • CPU count: 16

                         CONFIGURAZIONE PRONTA PE

In [16]:
import torch.nn.functional as F

# Directory per salvare i risultati
save_dir = "/home/brus/Projects/wavelet/elaborations/trained_models/"
os.makedirs(save_dir, exist_ok=True)

# Ottimizzatore
optimizer = SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

# Funzione di valutazione
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in loader:
            # Remove size check and interpolation
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# Funzione di addestramento
def train_epoch(model, loader, optimizer, epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(loader, desc=f'Epoch {epoch+1}')
    for data, target in progress_bar:
        # Sposta i dati sul device
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        # Aggiorna statistiche
        total_loss += loss.item() * target.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{correct/total:.4f}'
        })
    
    return total_loss / total, correct / total

# Debug info
first_batch, first_labels = next(iter(train_loader))
print(f"\nPrimo batch shape: {first_batch.shape}")
print(f"Numero di classi: {len(dataset.classes)}")
print(f"Classi: {dataset.classes}")
# Funzione per plottare le metriche
def plot_metrics(train_losses, train_accuracies, test_losses, test_accuracies, epoch, save=False):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot delle loss
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(test_losses, label='Test Loss')
    ax1.set_title('Loss over epochs')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Plot delle accuracy
    ax2.plot(train_accuracies, label='Train Accuracy')
    ax2.plot(test_accuracies, label='Test Accuracy')
    ax2.set_title('Accuracy over epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    
    if save:
        plt.savefig(f'{save_dir}/metrics_epoch_{epoch+1}.png')
        plt.close()
    else:
        plt.show()

# Funzione per caricare un checkpoint
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    Carica un checkpoint e restituisce il modello, l'ottimizzatore e le metriche
    """
    print(f"Caricamento checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    start_epoch = checkpoint['epoch'] + 1
    train_losses = checkpoint['train_losses']
    train_accuracies = checkpoint['train_accuracies']
    test_losses = checkpoint['test_losses']
    test_accuracies = checkpoint['test_accuracies']
    
    print(f"Checkpoint caricato dall'epoca {checkpoint['epoch']}")
    return model, optimizer, start_epoch, train_losses, train_accuracies, test_losses, test_accuracies


# Metriche per il monitoraggio
checkpoint_path = f"{save_dir}/model_epoch_4000.pt" 
if os.path.exists(checkpoint_path):
    model, optimizer, start_epoch, train_losses, train_accuracies, test_losses, test_accuracies = load_checkpoint(
        checkpoint_path, model, optimizer
    )
else:
    start_epoch = 0
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

# Training
print("Iniziando training...")
start_time = time.time()

for epoch in range(start_epoch, 90):  
    # Training
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, epoch)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # Validation
    test_loss, test_acc = evaluate(model, test_loader, device)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
    
    # Stampa risultati
    print(f'\nEpoch {epoch+1}/90:')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')
    
    # Plot ogni 10 epoche
    if (epoch + 1) % 10 == 0:
        plot_metrics(train_losses, train_accuracies, test_losses, test_accuracies, epoch)
        
        # Salva il checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses,
            'train_accuracies': train_accuracies,
            'test_losses': test_losses,
            'test_accuracies': test_accuracies,
        }, f'{save_dir}/model_epoch_{epoch+1}.pt')

# Tempo totale di training
total_time = time.time() - start_time
print(f'\nTraining completato in {total_time/60:.2f} minuti')

# Salva il modello finale
torch.save({
    'epoch': 90,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'test_losses': test_losses,
    'test_accuracies': test_accuracies,
}, f'{save_dir}/model_final.pt')

# Plot finale
plot_metrics(train_losses, train_accuracies, test_losses, test_accuracies, 89, save=True)
print(f'Modello e grafici salvati in: {save_dir}')


Primo batch shape: torch.Size([128, 3, 81, 8, 8])
Numero di classi: 7
Classi: ['human structure', 'mudflat', 'street', 'vegetation 1', 'vegetation 2', 'vegetation 3', 'water']
Iniziando training...


Epoch 1:   0%|          | 0/249 [00:00<?, ?it/s]


RuntimeError: Tensor must be of spatial size (32,32).

In [ ]:
# Funzione di inference
def classify_tile(model, tile_tensor):
    model.eval()
    with torch.no_grad():
        output = model(tile_tensor)
        probabilities = torch.softmax(output, dim=1)
        max_prob, label = torch.max(probabilities, dim=1)
        return label.item(), max_prob.item()